<a href="https://colab.research.google.com/github/aviaga/FutureMakers/blob/main/ML%20Secondary/Computer%20Vision%3A%20Emotion%20Detection%20through%20Facial%20Analysis%20(7-29).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Project Intro
This is the implementation of Day 24's Action Item for SureStart's summer program, 2021.

The goal of this project was to build a model that can identify a human's emotions through facial analysis.

The tutorial for this model development activity can be found [here](https://medium.com/swlh/emotion-detection-using-opencv-and-keras-771260bbd7f7).


# Model Creation

**Beginning steps: Package and Data imports**

In [ ]:
#Version install to match tutorial
#!pip install keras==2.3.1
#!pip install tensorflow==1.13.1


In [ ]:
#Importing libraries
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten,BatchNormalization
from tensorflow.keras.layers import Conv2D,MaxPooling2D
import os
from tensorflow.keras.optimizers import RMSprop,SGD,Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau


In [ ]:
#Defining some variables
num_classes=6
img_rows,img_cols=48,48
batch_size=32

In [ ]:
!unzip '/content/train-20210729T233616Z-001.zip'


Streaming output truncated to the last 5000 lines.
  inflating: train/Happy/264.jpg     
  inflating: train/Neutral/1436.jpg  
  inflating: train/Neutral/4214.jpg  
  inflating: train/Neutral/48.jpg    
  inflating: train/Neutral/1344.jpg  
  inflating: train/Neutral/74.jpg    
  inflating: train/Neutral/2895.jpg  
  inflating: train/Neutral/2117.jpg  
  inflating: train/Neutral/149.jpg   
  inflating: train/Fear/411.jpg      
  inflating: train/Neutral/808.jpg   
  inflating: train/Fear/3619.jpg     
  inflating: train/Neutral/2881.jpg  
  inflating: train/Neutral/3547.jpg  
  inflating: train/Fear/1807.jpg     
  inflating: train/Fear/439.jpg      
  inflating: train/Fear/3976.jpg     
  inflating: train/Fear/1152.jpg     
  inflating: train/Fear/377.jpg      
  inflating: train/Happy/6921.jpg    
  inflating: train/Neutral/834.jpg   
  inflating: train/Fear/1634.jpg     
  inflating: train/Happy/502.jpg     
  inflating: train/Neutral/3553.jpg  
  inflating: train/Fear/3745.jpg     

In [ ]:
!unzip /content/validation-20210729T233710Z-001.zip

Archive:  /content/validation-20210729T233710Z-001.zip
  inflating: validation/Angry/345.jpg  
  inflating: validation/Angry/387.jpg  
  inflating: validation/Angry/85.jpg  
  inflating: validation/Angry/0.jpg  
  inflating: validation/Angry/351.jpg  
  inflating: validation/Angry/190.jpg  
  inflating: validation/Angry/147.jpg  
  inflating: validation/Angry/219.jpg  
  inflating: validation/Angry/52.jpg  
  inflating: validation/Angry/218.jpg  
  inflating: validation/Neutral/46.jpg  
  inflating: validation/Happy/806.jpg  
  inflating: validation/Neutral/0.jpg  
  inflating: validation/Neutral/190.jpg  
  inflating: validation/Angry/153.jpg  
  inflating: validation/Angry/437.jpg  
  inflating: validation/Angry/46.jpg  
  inflating: validation/Angry/392.jpg  
  inflating: validation/Angry/350.jpg  
  inflating: validation/Angry/230.jpg  
  inflating: validation/Neutral/52.jpg  
  inflating: validation/Angry/379.jpg  
  inflating: validation/Angry/422.jpg  
  inflating: validation/An

In [ ]:
!unzip /content/train-20210729T233616Z-001.zip

Archive:  /content/train-20210729T233616Z-001.zip
replace train/.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
train_data_dir='/content/train'
validation_data_dir='/content/validation'

**Data Preprocessing**

In [ ]:
# Data Augmentation (Rotating, flipping, etc..) on training set 

train_datagen = ImageDataGenerator( rescale=1./255, rotation_range=30, shear_range=0.3, zoom_range=0.3, width_shift_range=0.4, height_shift_range=0.4, horizontal_flip=True, fill_mode='nearest')
validation_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
#Rescaling validation set
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    color_mode = 'grayscale',
    target_size = (img_rows,img_cols),
    batch_size = batch_size,
    class_mode = 'categorical',
    shuffle = True
)

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    color_mode = 'grayscale',
    target_size = (img_rows,img_cols),
    batch_size = batch_size,
    class_mode = 'categorical',
    shuffle = True
)

Found 28353 images belonging to 6 classes.
Found 3534 images belonging to 6 classes.


**Building the Model**

In [ ]:
#Defining model architecture 
model = Sequential()

#Layer 1
model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal',
                 input_shape=(img_rows,img_cols,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal',
                 input_shape=(img_rows,img_cols,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
#Layer 2
model.add(Conv2D(64,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(64,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
#Layer 3
model.add(Conv2D(128,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(128,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
#Layer 4
model.add(Conv2D(256,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(256,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
#Layer 5
model.add(Flatten())
model.add(Dense(64,kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
#Layer 6
model.add(Dense(64,kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
#Layer 7
#model.add(Flatten())
model.add(Dense(num_classes,kernel_initializer='he_normal'))
model.add(Activation('softmax'))

In [ ]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 48, 48, 32)        320       
_________________________________________________________________
activation (Activation)      (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 48, 48, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 48, 48, 32)        9248      
_________________________________________________________________
activation_1 (Activation)    (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 48, 48, 32)        128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 24, 24, 32)        0

**Compiling and Training the Model**

In [ ]:
#Creating some functions for better model training (for example, reducing the likelihood of overfitting)
checkpoint = ModelCheckpoint('EmotionDetectionModel.h5', monitor='val_loss', mode='min', save_best_only=True,verbose=1)
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3,verbose=1, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss',factor=0.2,patience=3,verbose=1,min_delta=0.0001)
callbacks = [earlystop,checkpoint,reduce_lr]

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer = Adam(lr=0.001),
              metrics=['accuracy'])

nb_train_samples = 24176
nb_validation_samples = 3006
epochs=25

history=model.fit_generator(
                train_generator,
              steps_per_epoch=nb_train_samples//batch_size,
                epochs=epochs,
                callbacks=callbacks,
                validation_data=validation_generator,
                validation_steps=nb_validation_samples//batch_size)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/25
755/755 [==============================] - 24s 30ms/step - loss: 2.1094 - accuracy: 0.1952 - val_loss: 1.7417 - val_accuracy: 0.2440

Epoch 00001: val_loss improved from inf to 1.74173, saving model to EmotionDetectionModel.h5
Epoch 2/25
755/755 [==============================] - 22s 29ms/step - loss: 1.7791 - accuracy: 0.2329 - val_loss: 1.7365 - val_accuracy: 0.2544

Epoch 00002: val_loss improved from 1.74173 to 1.73653, saving model to EmotionDetectionModel.h5
Epoch 3/25
755/755 [==============================] - 22s 29ms/step - loss: 1.7401 - accuracy: 0.2587 - val_loss: 1.7100 - val_accuracy: 0.2725

Epoch 00003: val_loss improved from 1.73653 to 1.71004, saving model to EmotionDetectionModel.h5
Epoch 4/25
755/755 [==============================] - 21s 28ms/step - loss: 1.7299 - accuracy: 0.2604 - val_loss: 1.6991 - val_accuracy: 0.2897

Epoch 00004: val_loss improved from 1.71004 to 1.69906, saving model to EmotionDetectionModel.h5
Epoch 5/25
755/755 [================